# REL 7: How does your system withstand component failures?

Predictable failure modes are considered for every component type.
Failure happen not only all the time, but in the most interesting ways. Although some outages are quite unpredictable, many of them can be expected, such as network partitioning, insufficient capacity, race conditions and many others. 

The most common and efficient failover mechanism is replacement with a redundant spare, particularly on the cloud where they can be instantly provisioned. To avoid a single point of failure, each component should be replaceable with minimal consequences, but that depends a lot on its architecture. Each system will handle failovers a bit differently according to its data management, but usually around one of the following strategies:

Multi-master services can receive requests in any node, like web servers for example, they all behave equally and, when they fail, clients will get the same responses from a different one without even noticing. That is usually the case for stateless services, but it gets more complicated when data needs to be replicated for redundancy. When a network failure happens and data can’t be replicated, services must choose between responding with errors or with stale data. In other words, when a network partition happens, distributed systems must choose between availability and consistency. This key principle is known as the CAP theorem and proposed by Eric Brewer in 1998. This decision depends on each business, but as outages can have significant implications, it is often easier to manage consistency. Given enough nodes and connections, algorithms such as Consistent Hashing, Vector Clocks and Sloppy Quorum can be used for designing distributed storage with minimal inconsistent states. Amazon DynamoDB uses these techniques to let you chose consistency mode in read time, so it’s up to you weather to get the first response or wait for consistency. But for customer-managed systems and other databases, replication type and latency need to be considered case by case.

If the replication group is within a single region, latency across Availability Zones are predictable and very low, typically less than a couple milliseconds. Then replication can be synchronous, waiting for copies to completed before sending a response to the client. It is as though the transaction happened at the same time in all servers and certainly consistent. This can be effective for a small number of servers, but as data grows, replication time can get larger than client timeout and lead to an outage.

If data is replicated to different regions, or to on-premises servers over the internet, latency can be unpredictable and too large to wait for synchronous replication. In these cases, it may be better keep the replication running asynchronously in the background and respond immediately to requests, even with stale data for a while.

Modern distributed systems may use a mix synchronous and asynchronous replication to reach the best of both worlds, but each will take a different approach. Databases such as DataStax Casandra uses “Multi-datacenter replication” and similar features to store data across the globe. When consistency is desired but not provided by the system, even with the posible availability issues, it is not uncommon to resort to a consistency tier, like Netflix S3mper does for S3 listings.

Unpredictable failure modes are investigated and prevented. Even the most resilient systems may eventually face an outage, due to bugs, human error and other unpredictable conditions. Sometimes, these conditions are so rare that it is not even worthy to work around them after fixing. But if that happens twice or more, it should be investigated and prevented, possibly with the same process and tools defined for security incident response (SEC 12).

